# Bing Search 기반의 RAG를 활용한 데모

이 데모에서는 GPT3 모델을 사용하여 자연 쿼리를 분석하고 지식 기반을 사용하여 더 많은 정보를 검색하고 질문에 답변하는 방법을 보여줍니다.  

해당 실습을 위해서는 [Bing 리소스](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.Bing%2Faccounts)를 만들고 API Key를 [.env](../.env) 파일에 입력해야 합니다. 

1. Bing Search API 활용: 검색한 결과를 참고하는 방법을 소개합니다. 여기서는 Bing Search API를 활용합니다.

In [1]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. Bing Search API를 활용하는 방법

### Start with a natural question  
다음과 같이 LLM이 가지고 있지 않을 수 있는 데이터에 대한 접근이 필요할 때, 검색엔진을 활용할 수 있습니다.

### Step 1: GPT3: What do I need to to answer this question?

In [2]:
# 검색을 위해 필요한 정보를 추출하는 함수 정의
def get_search_info(input_query):
    system_prompt='''
    Answer the following questions as best you can. You have access to the following tools: You can only Korean language.
    Web Search: Use the web to find information
    Question: Natural language-based questions that users want to ask through web searches
    Answer: Describes how to effectively answer people's natural language-based questions and creates optimized search keywords needed for use in the Bing Search API. And create optimized search keywords needed for use in the Bing Search API. The answer format must strictly follow the JSON format below.
    {"Keywords": "Specify three sets of keywords information to search for key data that matches the user's question intent."}
    '''
    user_prompt = f'''
    Question: {input_query}
    '''

    # Azure OpenAI GPT를 프롬프트 엔지니어링의 Few-shot 예시
    prompt = [{"role":"system", "content":system_prompt},
                {"role":"user", "content":"Question: 2024년 한국 총선 날짜가 언제인지 회사를 안 가도 되는지 알려줘."},
                {"role":"assistant", "content":"""{"Keywords": "2024년 한국 총선 날짜 + 2024년 총선 공휴일 여부 + 총선 투표일 휴무 정책"}"""},
                {"role":"user","content": user_prompt}
            ]
        
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
        max_tokens=600,
        response_format={"type": "json_object"}
    )

    keywords = json.loads(response.choices[0].message.content)['Keywords']
    
    # 만약 keywords가 list 형태일 경우, '+' 문자로 join하여 반환
    if type(keywords) == list:
        keywords = ' + '.join(keywords)

    return json.loads(response.choices[0].message.content)['Keywords']

### Step 2: Search web for more details  

Bing Search API와 관련된 정보는 다음 Page에서 확인할 수 있습니다.  
https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

In [3]:
# Bing search API를 이용하여 검색 결과를 가져오는 함수 정의
def get_search_result(keywords):
    subscription_key = os.getenv("BING_SEARCH_KEY")
    endpoint = os.getenv("BING_SEARCH_ENDPOINT") + "/v7.0/search"

    query = keywords

    # Construct a request
    mkt = 'ko_KR'
    params = { 'q': query, 'mkt': mkt }
    headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)

    context = ''
    for result in response.json()['webPages']['value']:
        context += result['snippet'] + '\n'

    return context

In [4]:
# 검색 결과를 기반으로 사용자의 질문에 대한 답을 생성하는 함수 정의
def generate_answer(input_query, context):
    system_prompt = """You are an agent who answers the user's questions based on the data provided. You can only Korean language."""
    user_prompt = f"""Question: {input_query}
    Web Search Results: {context}
    Answer: 
    """

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user","content": user_prompt}
        ],
        max_tokens=600
    )

    return response.choices[0].message.content

In [5]:
# 사용자로부터 검색 질의어를 입력받아서 결과를 출력하는 함수 정의
def get_input_query(input_query):
    # 검색 질의어 출력
    print("-"*80)
    print("검색질의어: " + input_query)
    print("-"*80)

    # 검색 키워드를 추출하는 함수 호출
    keywords = get_search_info(input_query)
    print("검색키워드: " + keywords)
    print("-"*80)

    # 검색 결과를 가져오는 함수 호출
    context = get_search_result(keywords)
    print("검색결과: " + context)
    print("-"*80)

    # 사용자에게 제공할 답변을 생성하는 함수 호출
    answer = generate_answer(input_query, context)
    print("최종답변: " + answer)
    print("-"*80)

In [6]:
input_query = "2024년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?"
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2024년 '부처님 오신 날'이 언제야? 쉬는 날 맞아? 공휴일이면 대체 휴무는 언제야?
--------------------------------------------------------------------------------
검색키워드: 2024년 부처님 오신 날 날짜 + 2024년 부처님 오신 날 공휴일 여부 + 2024년 부처님 오신 날 대체 휴무
--------------------------------------------------------------------------------
검색결과: 2024년 부처님 오신 날 <날짜>. 2024년 부처님오신날 법정공휴일. 2024년 부처님 오신 날은 불교의 연중행사 가운데 가장 큰 명절로서, 많은 사람들이 부처의 탄생을 기념하며 기념법회, 연등 행사 등, 각종 기념행사를 주최하고 참석합니다. 대한민국뿐만 ...
양력으로 고정되어 있는 날이 아니기 때문에 해마다 날짜가 달라집니다. 매해 부처님 오신날이 언제인지 확인해야 합니다. 올해 24년 부처님 오신날은 양력으로 5월 15일입니다. 5월 15일은 스승의 날인데라고 생각하신 분들도 많을 것 같습니다. 스승의 날은 공휴일이 아니라서 은사님을 찾아뵙지 못하는 경우가 많습니다. 하지만 올해는 공휴일인 부처님 오신날과 겹쳤기 때문에 은사님을 직접 찾아뵐 수 있을 것 같습니다. 2024년 부처님 오신날에는 사찰에 방문하거나, 은사님께 감사하는 마음을 표현하거나 둘 중 하나를 선택하여 하면 좋을 것 같습니다.
1. 부처님오신날 날짜. - 2024년 5월 15일. 공교롭게도 스승의날 (5월 15일)과 겹치는데요. 아시는 것처럼 부처님오신날은 '음력'으로 세기 때문에 매년 바뀌게 됩니다. 2025년 부처님오신날은 2025년 5월 5일입니다. 2. 2025년 5월 달력. 미리 보여 드리면, 2025년 5월은 

In [7]:
input_query = "2024년 추석 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘."
get_input_query(input_query)

--------------------------------------------------------------------------------
검색질의어: 2024년 추석 연휴는 언제야? 연휴 기간은으로 얼만큼 쉴 수 있을지 알려줘.
--------------------------------------------------------------------------------
검색키워드: 2024년 추석 날짜 + 2024년 추석 연휴 기간 + 2024년 추석 공휴일
--------------------------------------------------------------------------------
검색결과: 우리는 현재 2024년 공휴일을 목록으로 가지고 있습니다. 공휴일을 확인하시려면 스크롤을 내려주세요. 인기있는 휴일: 설날. 추석. 현충일.
2024년 추석 날짜 2024년-추석. 2024년 추석 날짜는 9월 17일, 화요일입니다. 추석연휴는 주말과 겹치지 않아서 총 5일의 긴 휴일이 가능할 예정이에요. 올 해 추석은 약간 빠른 것 같네요. 2024년 추석: 9월 17일; 2024년 추석 연휴: 9월 14일 토요일 – 9월 18일 ...
또한 추석은 9월 16일 월요일부터 9월 17일 화요일, 9월 18일 수요일이기 때문에 대체 휴일 없이 온전히 휴무를 즐길 수 있습니다. 👇 자세한 정보는 아래에서 확인하세요! 네이버 캘린더 설치하기. 2024년 대한민국 공휴일 황금연휴 기간. 황금 연휴 기간은 아무래도 모든 사람들이 기다리고, 또 기다리는 시간이 아닐까 싶을 만큼 많은 분들께서 연차를 사용하지 않고도 편하게 쭉 쉴 수 있는 연휴 기간이기도 한데요. 올해는 아쉽게도 일주일 넘게 쭉 쉴 수 있는 날이 흔하지도 않을 뿐더러 추석이나 설날과 같은 날이 아니라면 평일 하루 정도 쉬는 것에 그치기 때문에 많이 아쉬운 부분이 남기도 합니다.
일요일 ~ 화요일. 추석. 2026. 9월 24일 ~ 9월 27일. 목요일 ~ 일요일. 추석. 이전 연도의 날짜를 